In [ ]:
# ! pip3 install louvain --user

In [ ]:
# !python -m pip install community

# Import Libraries and load files

In [1]:
import os 
import glob
import sklearn.cluster

circles_files = glob.glob("twitter/*.circles")
edges_files = glob.glob("twitter/*.edges")
egofeat_files = glob.glob("twitter/*.egofeat")
feat_files = glob.glob("twitter/*.feat")
featnames_files = glob.glob("twitter/*.featnames")

In [2]:
import numpy

In [3]:
from nf1 import NF1

In [4]:
import os
from collections import defaultdict
import random
import sys
from munkres import Munkres
import numpy
%matplotlib inline
import networkx as nx

In [6]:
import community

In [7]:
from sklearn.cluster import SpectralClustering

In [22]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Build a Graph

In [9]:
def read_nodeadjlist(filename):
    G = nx.Graph()
    for line in open(filename):
        lst = line.split(' ')
        el, es = lst[0], lst[1][:-1]
        G.add_edge(int(el),int(es))
    return G

# Define Loss function

In [21]:
def loss1(usersPerCircle, usersPerCircleP):
    psize = max(len(usersPerCircle),len(usersPerCircleP)) 
    mm = numpy.zeros((psize,psize))
    mm2 = numpy.zeros((psize,psize))
    for i in range(psize):
        for j in range(psize):
            circleP = set() 
            circle = set() 
            if (i < len(usersPerCircleP)):
                circleP = usersPerCircleP[i]
            if (j < len(usersPerCircle)):
                circle = usersPerCircle[j]
            nedits = len(circle.union(circleP)) - len(circle.intersection(circleP)) 
            mm[i][j] = nedits
            mm2[i][j] = nedits

    if psize == 0:
        return 0 
    else:
        m = Munkres()
        indices = m.compute(mm) 
        editCost = 0
        for row, column in indices:
            editCost += mm2[row][column]
    return int(editCost)


# Calculate Similarity between users of one egonet by constructing feature vector from # and @ followers feature list 

In [17]:

def read_features_trial_2(featname):
    features = {}
    featnames = []
    nodeIdSim = {}
    file= open(featname)
    file_lst = file.readlines()
    fin_rslt = [[None for i  in range(len(file_lst))] for j in range(len(file_lst))]
    for i in range(len(file_lst)):
        lst = file_lst[i].split(" ")
        nodeId1,fts1 = lst[0],lst[1:]
        fts1 = np.asarray(fts1)
        fts1 = fts1.reshape(1,-1) 
        j = 0
        while(j < len(file_lst)):
            nodeIdRel = ()
            lst2 = file_lst[j].split(' ')            
            nodeId2,fts2 = lst2[0],lst2[1:]
            nodeIdRel=(nodeId1,nodeId2)
            fts2 = np.asarray(fts2)
            fts2 = fts2.reshape(1,-1)
            similarity = cosine_similarity(fts1,fts2)
            nodeIdSim[nodeIdRel] = similarity[0][0]
            fin_rslt[i][j] = similarity[0][0]
            j+=1
    return fin_rslt

In [ ]:
# user_feat_2 = read_features_trial_2("twitter/237845487.feat")

In [ ]:
# user_feat_2

In [ ]:
# cl = sklearn.cluster.SpectralClustering(affinity="precomputed",n_clusters=15)

In [ ]:
# cl.fit(user_feat_2)

In [ ]:
# cl.labels_

In [ ]:
# c2 = sklearn.cluster.spectral_clustering(nx.adjacency_matrix(G))


In [ ]:
# c2

# Define Spectral Clustering 

In [10]:
def naive_spec_cluster(G,ego):
    cl = sklearn.cluster.SpectralClustering(affinity="precomputed")
    user_feat = read_features_trial_2("twitter/"+ego+".feat")
    cl.fit(user_feat)
    pred_circles ={}
    for circle,user in zip(cl.labels_,G.nodes()):
        if circle in pred_circles:
            pred_circles[circle].append(user)
        else:
            pred_circles[circle]=[user]
    return pred_circles

# Create List of egos[973 ego list]

In [11]:
egoNodeList = []
for item in circles_files:
    twitter, circleFilename = item.split("/")
    filename, abcd = circleFilename.split(".")
    egoNodeList.append(filename)

# Filter list of egonet on basis of number of edges present in respective edge file

In [12]:
filterNodeList = []
num_lines_len = []
for item in edges_files:
    num_lines = sum(1 for line in open(item))
    num_lines_len.append(num_lines)
    if num_lines < 2300:
        twitter, circleFilename = item.split("/")
        filename, abcd = circleFilename.split(".")
        filterNodeList.append(filename)

In [13]:
len(egoNodeList)

973

In [14]:
len(filterNodeList)

583

# Read Ground Truth Circle files

In [15]:
def read_circles(filename):
    final_lst = []
    for line in open(filename):
        lst = line.split('\t')
        el, es = lst[0], lst[1:]
        circle  =set()
        for e in es:
            circle.add(int(e))
        final_lst.append(circle)
    return final_lst

# Calculate Modularity and loss of Ego Nets

In [24]:
Final_op = {}
cnt = 0
totalLoss = 0
Modularity_map = {}
for ego in filterNodeList:
    G = read_nodeadjlist("twitter/"+ego+".edges")
    pred_circles = naive_spec_cluster(G,ego)
    Final_op[ego] = pred_circles
    part = community.best_partition(G)
    mod = community.modularity(part,G)
    Modularity_map[ego] = mod
    pred = []
    for key, value in pred_circles.items():
        pred.append(set(value))
    gt_circles = read_circles("twitter/"+ego+".circles")
    calLoss = loss1(gt_circles, pred)
    print(calLoss)
    totalLoss +=calLoss
    cnt+=1
    print(cnt)
print(totalLoss)

22
1


/opt/ohpc/pub/apps/python3/3.7.0/lib/python3.7/site-packages/sklearn/manifold/spectral_embedding_.py:234: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"


85
2
72
3
33
4
153
5
114
6
227
7
63
8
145
9
133
10
168
11
107
12
40
13
82
14
109
15
46
16
52
17
37
18
105
19
67
20
85
21
128
22
89
23
131
24
30
25
137
26
77
27
142
28
92
29
102
30
108
31
103
32
93
33
92
34
76
35
89
36
130
37
59
38
37
39
160
40
68
41
203
42
261
43
102
44
97
45
121
46
40
47
85
48
168
49
180
50
12
51
77
52
211
53
83
54
113
55
44
56
152
57
76
58
76
59
118
60
111
61
16
62
14
63
87
64
92
65
72
66
207
67
101
68
15
69
80
70
148
71
138
72
96
73
78
74
35
75
67
76
33
77
262
78
13
79
89
80
157
81
75
82
63
83
148
84
159
85
142
86
183
87
176
88
169
89
65
90
176
91
39
92
149
93
198
94
130
95
206
96
173
97
84
98
93
99
137
100
52
101
102
102
136
103
28
104
48
105
220
106
235
107
113
108
129
109
66
110
121
111
56
112
97
113
44
114
81
115
116
116
87
117
70
118
112
119
63
120
25
121
124
122
82
123
26
124
22
125
76
126
61
127
164
128
187
129
63
130
103
131
101
132
115
133
70
134
120
135
110
136
212
137
32
138
168
139
88
140
34
141
125
142
88
143
91
144
88
145
150
146
84
147
14
148
87
149
1

In [25]:
print(totalLoss)

63764


In [35]:
AvgLoss = totalLoss /583

In [36]:
print(AvgLoss)

109.37221269296741


In [38]:
modularity_sum = sum(Modularity_map.values())

In [40]:
AvgModularity = modularity_sum/583

In [41]:
AvgModularity

0.30444641348842943

In [ ]:
# Final_op = {}
# cnt = 0
# totalLoss = 0
# Modularity_map = {}
# for ego in filterNodeList:
#     G = read_nodeadjlist("twitter/"+ego+".edges")
#     pred_circles = naive_spec_cluster(G,ego)
#     Final_op[ego] = pred_circles
#     pred = []
#     for key, value in pred_circles.items():
#         pred.append(set(value))
#     gt_circles = read_circles("twitter/"+ego+".circles")
#     calLoss = loss1(gt_circles, pred)
#     print(calLoss)
#     totalLoss +=calLoss
#     cnt+=1
#     print(cnt)
# print(totalLoss)

# Calculate Normalized F1 on Spectral Clustering 

In [18]:
Final_op = {}
f1_result_map ={}
f1_result_val = []
cnt = 0
for ego in filterNodeList:
    G = read_nodeadjlist("twitter/"+ego+".edges")
    pred_circles = naive_spec_cluster(G,ego)
    Final_op[ego] = pred_circles
    pred = []
    for key, value in pred_circles.items():
        pred.append(set(value))
    gt_circles = read_circles("twitter/"+ego+".circles")
    if pred and gt_circles:
        nf = NF1(pred,gt_circles)
        results = nf.summary()
        df = results['scores']
        nf1 = df["Value"][5]
        f1_result_map[ego] = nf1
        f1_result_val += nf1
        cnt+=1
        print(cnt)


1


/opt/ohpc/pub/apps/python3/3.7.0/lib/python3.7/site-packages/sklearn/manifold/spectral_embedding_.py:234: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"


2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
27

In [30]:
f1_result_map

{'333881828': 0.081875,
 '21318259': 0.011875000000000002,
 '2257971': 0.044,
 '120157205': 0.044375,
 '45857296': 0.132,
 '93998005': 0.060000000000000005,
 '19760380': 0.013593750000000002,
 '40022380': 0.024583333333333332,
 '253751053': 0.13125000000000003,
 '15347317': 0.0134375,
 '39157827': 0.07875,
 '61890555': 0.07875000000000001,
 '248523038': 0.0215,
 '18201401': 0.04875,
 '119897041': 0.059375,
 '27595997': 0.011625,
 '167901620': 0.03025,
 '83883736': 0.07833333333333334,
 '74264908': 0.0072499999999999995,
 '35166642': 0.04785714285714286,
 '1046661': 0.028526785714285713,
 '14827526': 0.00375,
 '30824363': 0.019375000000000003,
 '356897757': 0.013888888888888888,
 '12152402': 0.0325,
 '11828432': 0.007569444444444445,
 '37723441': 0.047727272727272736,
 '26346966': 0.08390625,
 '77843323': 0.10892045454545456,
 '28628055': 0.05375,
 '33080832': 0.021249999999999998,
 '168537920': 0.08000000000000002,
 '51775432': 0.018392857142857138,
 '14372486': 0.01648148148148148,
 '

In [31]:
sum_f1 = sum(f1_result_map.values())

In [42]:
sum_f1

30.945911066296347

In [33]:
avgF1 = sum_f1/583

In [34]:
avgF1

0.053080464950765606